In [13]:
# %pip list |grep keras

keras                         2.7.0
Note: you may need to restart the kernel to use updated packages.


In [12]:
# %pip install keras==2.7.0

  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.7.0 requires numpy>=1.14.5, which is not installed.
Note: you may need to restart the kernel to use updated packages.


## Imports

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
from tensorflow.keras import layers

print("TensorFlow version:", tf.__version__)
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#     try:
#         # Currently, memory growth needs to be the same across GPUs
#         for gpu in gpus:
#             tf.config.experimental.set_memory_growth(gpu, True)
#         logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#         print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#     except RuntimeError as e:
#         # Memory growth must be set before GPUs have been initialized
#         print(e)

TensorFlow version: 2.8.0


In [2]:
import pathlib
import sys


sys.path.insert(0,'..')
from src.load_dataset_fft_aug import get_splitter_dataloaders_fft

# Build model

In [10]:
model = tf.keras.models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation="relu", input_shape=(201, 498)),
    layers.MaxPool1D(pool_size=4, strides=4),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.MaxPool1D(pool_size=2, strides=2),

    layers.Conv1D(64, kernel_size=3, activation="relu"),
    layers.Dropout(0.5),
    
    layers.LSTM(units=40, return_sequences=True),
    
    layers.MaxPool1D(pool_size=2, strides=2),
    layers.Flatten(),
    
    layers.Dense(2)
    ])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# L1 loss
model.compile(optimizer=optimizer,
            loss="mae",
            metrics=["mae"]
            )

In [4]:
import numpy as np
x = tf.random.normal([1, 201, 498])

In [5]:
model.fit(x=x, y=tf.constant([[0.0, 0.0]]), epochs=1)

1/1 [==============================] - 2s 2s/step - loss: 0.3850 - mae: 0.3850


### Model Training

Bring dataset

In [6]:
_, _, train, val = get_splitter_dataloaders_fft(noise_attenuation=0.00001)

Caching dataset: 100%|██████████| 1144/1144 [00:06<00:00, 180.60it/s]


Build a generator for the dataset in order to feed it to the model directly.

In [7]:
# y_train = []
# for i in range(len(train)):
#     y_train.append(train[i][1])

# y_val = []
# for i in range(len(val)):
#     y_val.append(val[i][1])

In [11]:
def train_generator():
    for i in range(len(train)):
        instance = train[i]
        yield instance[0], instance[1]
        
def val_generator():
    for i in range(len(val)):
        instance = val[i]
        yield instance[0], instance[1]
        
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_generator(train_generator, output_types=(tf.float32, tf.float32), output_shapes=((201, 498), (2,))).padded_batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_generator(val_generator, output_types=(tf.float32, tf.float32), output_shapes=((201, 498), (2,))).padded_batch(BATCH_SIZE)

Fit the model 😃

In [12]:
model.fit(train_dataset, epochs=50, validation_data=val_dataset)

Epoch 1/50
72/72 [==============================] - 7s 94ms/step - loss: 1.6679 - mae: 1.6679 - val_loss: 1.3256 - val_mae: 1.3256
Epoch 2/50
72/72 [==============================] - 6s 81ms/step - loss: 1.2206 - mae: 1.2206 - val_loss: 1.1131 - val_mae: 1.1131
Epoch 3/50
72/72 [==============================] - 6s 80ms/step - loss: 1.1215 - mae: 1.1215 - val_loss: 0.9988 - val_mae: 0.9988
Epoch 4/50
72/72 [==============================] - 6s 81ms/step - loss: 1.0530 - mae: 1.0530 - val_loss: 0.9864 - val_mae: 0.9864
Epoch 5/50
72/72 [==============================] - 6s 80ms/step - loss: 1.0116 - mae: 1.0116 - val_loss: 0.9055 - val_mae: 0.9055
Epoch 6/50
72/72 [==============================] - 6s 82ms/step - loss: 0.9617 - mae: 0.9617 - val_loss: 0.8653 - val_mae: 0.8653
Epoch 7/50
72/72 [==============================] - 6s 80ms/step - loss: 0.9363 - mae: 0.9363 - val_loss: 0.8529 - val_mae: 0.8529
Epoch 8/50
72/72 [==============================] - 6s 80ms/step - loss: 0.8873 - m

In [12]:
model.fit(val_dataset, epochs=2)

Epoch 1/2
18/18 [==============================] - 1s 60ms/step - loss: 0.6581 - mae: 0.6581
Epoch 2/2
18/18 [==============================] - 1s 54ms/step - loss: 0.6253 - mae: 0.6253


### Save model

In [13]:
save_path = "saved/tf_model"
# tf.saved_model.save(model, save_path)

run_model = tf.function(lambda x: model(x))
BATCH_SIZE = 1
FREQS = 201
TIME_STEPS = 498
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, FREQS, TIME_STEPS], model.inputs[0].dtype))
model.save(save_path, save_format="tf", signatures=concrete_func)

INFO:tensorflow:Assets written to: saved/tf_model/assets


INFO:tensorflow:Assets written to: saved/tf_model/assets


# Tinify model

In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model(save_path)

save_path_tflite = os.path.basename(save_path) + ".tflite"

converter.optimizations = [tf.lite.Optimize.DEFAULT]


tflite_model = converter.convert()
tflite_model_file = pathlib.Path(save_path_tflite)
tflite_model_file.write_bytes(tflite_model)

Estimated count of arithmetic ops: 20.144 M  ops, equivalently 10.072 M  MACs


97776

# Test inference

This cell is for testing the model without rerunning the cells above.

In [6]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved/tf_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

In [26]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

interpreter.set_tensor(input_index, tf.random.normal([1, 201, 498]))

interpreter.invoke()
output = interpreter.tensor(output_index)()[0]
output

array([-0.42965645,  0.11463061], dtype=float32)